In [1]:
#initialize
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision 
import torchvision.transforms as transforms 
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
 
import time
import matplotlib
import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

 #hyperparameters data size 28*28*1, 10 classes
num_classes = 10
in_channel = 1
learning_rate = 0.001
batch_size = 128
num_epochs = 30

In [2]:
from torchvision import models

resnet18 = models.resnet18(pretrained=True)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /root/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


In [3]:
class ResNet(nn.Module):
  def __init__(self, in_channels, resnet18, num_classes):
    super(ResNet, self).__init__()
    self.model = resnet18
    self.model.conv1 = nn.Conv2d(1, 64, 3)
    self.model.fc = nn.Linear(512, num_classes)

  def forward(self, x):
    x = self.model(x)
    return x

In [4]:
model = ResNet(in_channel,resnet18,num_classes).to(device)
#Loss, Optimizer, LR scheduler
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adagrad(model.parameters(), lr=learning_rate, weight_decay=0.1)

In [5]:
transforms_transform = transforms.Compose([transforms.ToTensor(),
                                transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

class DataSet(Dataset):
  def __init__(self, train_x, train_y, transform=transforms_transform):
    self.X = train_x
    self.y = train_y
    self.transform = transform      

  def __getitem__(self, item):
    x_t = torch.Tensor(self.X[item])
    y_t = self.y[item]
    return (x_t, int(y_t))
  def __len__(self):
    return len(self.X)

In [6]:
from google.colab import drive 

drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [8]:
#load test_data from csv

csv_test ='/content/AIcomm_project_1_test.csv' 

cols = list(pd.read_csv(csv_test, nrows =1))
scaler = StandardScaler()

csv_test_features = pd.read_csv(csv_test, index_col = 0, usecols =[i for i in cols if i != 'label'])
csv_test_labels = pd.read_csv(csv_test, index_col = 0, usecols=[0, 1])

test_features = csv_test_features.values
test_features = scaler.fit_transform(test_features)

test_labels = csv_test_labels.values
"""test_labels = test_labels.reshape(-1)"""

test_features = test_features.reshape(10000,1,28,28)

dataset_test = DataSet(test_features, test_labels)

test_loader = DataLoader(dataset = dataset_test, batch_size = batch_size, shuffle = False)

In [ ]:
#load trained model and evaluate model on test data
test_model = ResNet(1,resnet18,10).to(device)
test_model.load_state_dict(torch.load('/content/gdrive/MyDrive/Fassion_Classification.pth'))
test_model.eval()
with torch.no_grad():
  correct = 0
  for image, label in test_loader:
    image = image.to(device)
    label = label.to(device)
    out = model(image)
    _, pred = torch.max(out.data, 1)
    correct += (pred == label).sum().item()
  print("Accuracy of the network on the {} test images: {}%".format(len(test_loader)*batch_size, 100 * correct / (len(test_loader) * batch_size)))